In [2]:
import streamlit as st
import pandas as pd
import altair as alt
import numpy as np

In [3]:
f1 = pd.read_csv('data/chefmozaccepts.csv',encoding='ISO-8859-1')

In [4]:
files = ['chefmozaccepts',
         'chefmozcuisine',
         'chefmozhours4',
         'chefmozparking',
         'usercuisine',
         'userpayment',
         'userprofile',
         'rating_final',
         'geoplaces2']
dfs = dict()

for file in files:
    fl= 'data/'+file+'.csv'
    dfs[file] = pd.read_csv(fl,encoding='ISO-8859-1')
    alt.data_transformers.disable_max_rows()

In [58]:
dfs['chefmozhours4']['hours'] = dfs['chefmozhours4']['hours'].str.strip()
dfs['chefmozhours4']['days'] = dfs['chefmozhours4']['days'].str.strip()


h1 = dfs['chefmozhours4']['hours'].str.split('-',expand = True)
h1[0] = h1[0].astype('datetime64[ns]')
h1[0] = pd.to_datetime(h1[0], format='%d/%m/%y %H:%M')
h2= h1[1].str.split(';',expand = True)
h1[1]= h2[0].astype('datetime64[ns]')
h1[1] = pd.to_datetime(h1[1], format='%d/%m/%y %H:%M')
h1['openhr'] = h1[0].dt.hour
h1['openmin'] = h1[0].dt.minute
h1['clshr'] = h1[1].dt.hour
h1['clsmin'] = h1[1].dt.minute

dfs['chefmozhours4']['opentime'] =h1[0].dt.time
dfs['chefmozhours4']['openhr'] = h1['openhr']
dfs['chefmozhours4']['openmin'] = h1['openmin']
dfs['chefmozhours4']['clstime'] =h1[1].dt.time
dfs['chefmozhours4']['clshr'] = h1['clshr']
dfs['chefmozhours4']['clsmin'] = h1['clsmin']

#converting days of operation into corresponding column

d1 =dfs['chefmozhours4']['days'].str.split(';',expand = True)
d1.drop([5],axis=1,inplace=True)
d1['Mon'] = d1[0] .apply(lambda x: (x=='Mon')==True )
d1['Tue'] = d1[1] .apply(lambda x: (x=='Tue')==True )
d1['Wed'] = d1[2] .apply(lambda x: (x=='Wed')==True )
d1['Thu'] = d1[3] .apply(lambda x: (x=='Thu')==True )
d1['Fri'] = d1[4] .apply(lambda x: (x=='Fri')==True )
d1['Sat'] = d1[0] .apply(lambda x: (x=='Sat')==True )
d1['Sun'] = d1[0] .apply(lambda x: (x=='Sun')==True )
d1.replace({False: 0, True: 1}, inplace=True)
d1['days_of_operation'] = d1['Mon']+d1['Tue']+d1['Wed']+d1['Thu']+d1['Fri']+d1['Sat']+d1['Sun']

dfs['chefmozhours4']['Mon'] = d1['Mon']
dfs['chefmozhours4']['Tue'] = d1['Tue']
dfs['chefmozhours4']['Wed'] = d1['Wed']
dfs['chefmozhours4']['Thu'] = d1['Thu']
dfs['chefmozhours4']['Fri'] = d1['Fri']
dfs['chefmozhours4']['Sat'] = d1['Sat']
dfs['chefmozhours4']['Sun'] = d1['Sun']
dfs['chefmozhours4']['days_of_operation'] =  d1['days_of_operation']


#drop res_df if exists

# for col in res_df:
#     res_df.drop(col,axis=1,inplace=True)
    
#Merge the files to get the rating file
res_df = dfs['chefmozaccepts'].merge(dfs['chefmozcuisine'],on ='placeID' )
res_df = res_df.merge(dfs['chefmozhours4'],on ='placeID' )
res_df = res_df.merge(dfs['chefmozparking'],on ='placeID' )
res_df = res_df.merge(dfs['geoplaces2'],on ='placeID' )
res_rating = res_df.merge(dfs['rating_final'],on ='placeID' )
usr_df = dfs['usercuisine'].merge(dfs['userpayment'],on ='userID' )
usr_df = usr_df.merge(dfs['userprofile'],on ='userID' )
usr_rating = usr_df.merge(dfs['rating_final'],on ='userID' )

s1=["SLP","S.L.P.","s.l.p.","San Luis Potosi","slp","san luis potos","san luis potosi"]
s2 =["tamaulipas","Tamaulipas"]
s3=["morelos","Morelos"]
s4=["?","mexico"]
res_rating['state'] =  res_rating['state'].apply(lambda x: "San Luis Potosi" if x in s1 else x)
res_rating['state'] =  res_rating['state'].apply(lambda x: "Tamaulipas" if x in s2 else x)
res_rating['state'] =  res_rating['state'].apply(lambda x: "Morelos" if x in s3 else x)
res_rating['state'] =  res_rating['state'].apply(lambda x: "Not Determined" if x in s4 else x)

# creating Ranking dataframe for rating chart
filt_cols = ['placeID','name' ,'rating','food_rating','service_rating','state','price','parking_lot']
ranking = pd.DataFrame(res_rating,columns=filt_cols)
ranking['Avg_Rating'] = ranking[['rating','food_rating','service_rating']].mean(axis=1)
ranking = ranking[ranking['state'] != 'Not Determined' ]
ranking['parking_lot'] =  ranking['parking_lot'].apply(lambda x: "Not available" if x in 'none' else x)


C:\Users\LakuJayanth\AppData\Local\Temp\ipykernel_4680\3305925977.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  h1[0] = h1[0].astype('datetime64[ns]')
C:\Users\LakuJayanth\AppData\Local\Temp\ipykernel_4680\3305925977.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  h1[1]= h2[0].astype('datetime64[ns]')


In [55]:
g

,name,food_rating,service_rating,state,price,parking_lot,rating
350,tacos los volcanes,2,2,San Luis Potosi,low,Not available,2.0
250,Restaurante Tiberius,2,0,San Luis Potosi,medium,public,2.0
248,Restaurante Tiberius,1,0,San Luis Potosi,medium,public,2.0
247,Restaurante Tiberius,0,2,San Luis Potosi,medium,public,2.0
116,Koye Sushi,1,0,San Luis Potosi,high,Not available,2.0
117,Koye Sushi,1,1,San Luis Potosi,high,Not available,2.0
118,Koye Sushi,2,0,San Luis Potosi,high,Not available,2.0
119,Koye Sushi,2,2,San Luis Potosi,high,Not available,2.0
124,La Cantina Restaurante,0,2,San Luis Potosi,high,Not available,2.0
125,La Cantina Restaurante,1,0,San Luis Potosi,high,Not available,2.0


In [32]:
res_rating.columns

Index(['placeID', 'Rpayment', 'Rcuisine', 'hours', 'days', 'opentime',
       'openhr', 'openmin', 'clstime', 'clshr', 'clsmin', 'Mon', 'Tue', 'Wed',
       'Thu', 'Fri', 'Sat', 'Sun', 'days_of_operation', 'parking_lot',
       'latitude', 'longitude', 'the_geom_meter', 'name', 'address', 'city',
       'state', 'country', 'fax', 'zip', 'alcohol', 'smoking_area',
       'dress_code', 'accessibility', 'price', 'url', 'Rambience', 'franchise',
       'area', 'other_services', 'userID', 'rating', 'food_rating',
       'service_rating'],
      dtype='object')

In [84]:
# res_rating[(res_rating['rating'] > 0) & (res_rating['state'] == 'San Luis Potosi')]
rs_cus_list = (res_rating[(res_rating['rating'] > 0) & (res_rating['state'] == 'San Luis Potosi') &(res_rating['state'] == 'San Luis Potosi')]['Rcuisine'].unique())
# rs_st_list = rs_st_list.tolist()
# rs_st_list.remove('Not Determined') 
rs_cus_list

array(['Mexican', 'Burgers', 'Fast_Food', 'Chinese', 'Seafood',
       'Contemporary', 'Bar', 'Bar_Pub_Brewery', 'Japanese', 'Pizzeria',
       'Cafeteria', 'Italian', 'Cafe-Coffee_Shop', 'American',
       'Breakfast-Brunch', 'Bakery', 'International', 'Game'],
      dtype=object)

In [147]:
res_rating['alcohol'].value_counts()

alcohol
No_Alcohol_Served    3204
Wine-Beer            2121
Full_Bar              951
Name: count, dtype: int64

In [108]:
cus_list =['Bar', 'Bar_Pub_Brewery', 'Japanese']
bool(cus_list)

True

In [90]:
res_rat_filter  = res_rating[(res_rating['rating'] > 0) & (res_rating['state'] == 'San Luis Potosi')& (res_rating['Rcuisine'].isin(cus_list) )]

In [166]:
res_rat_filter  = res_rating[(res_rating['rating'] > 0) & (res_rating['state'] == 'San Luis Potosi')& (res_rating['Rcuisine'].isin(cus_list) )&(res_rating['parking_lot']!= 'none' )]


In [109]:
if bool(cus_list):
    
    res_rat_filter  = res_rating[(res_rating['rating'] > 0) & (res_rating['state'] == 'San Luis Potosi')& (res_rating['Rcuisine'].isin(cus_list) )]
else:
    res_rat_filter  = res_rating[(res_rating['rating'] > 0) & (res_rating['state'] == 'San Luis Potosi')]
    
    res_rat_filter

In [110]:
res_rat_filter

,placeID,Rpayment,Rcuisine,hours,days,opentime,openhr,openmin,clstime,clshr,...,price,url,Rambience,franchise,area,other_services,userID,rating,food_rating,service_rating
648,135073,cash,Bar,09:30-23:30;,Mon;Tue;Wed;Thu;Fri;,09:30:00,9,30,23:30:00,23,...,high,?,familiar,f,closed,none,U1076,2,1,1
649,135073,cash,Bar,09:30-23:30;,Mon;Tue;Wed;Thu;Fri;,09:30:00,9,30,23:30:00,23,...,high,?,familiar,f,closed,none,U1022,2,2,2
650,135073,cash,Bar,09:30-23:30;,Mon;Tue;Wed;Thu;Fri;,09:30:00,9,30,23:30:00,23,...,high,?,familiar,f,closed,none,U1024,1,2,0
651,135073,cash,Bar,09:30-23:30;,Mon;Tue;Wed;Thu;Fri;,09:30:00,9,30,23:30:00,23,...,high,?,familiar,f,closed,none,U1101,1,1,1
652,135073,cash,Bar,09:30-23:30;,Sat;,09:30:00,9,30,23:30:00,23,...,high,?,familiar,f,closed,none,U1076,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4480,132937,cash,Bar,21:00-02:00;,Sat;,21:00:00,21,0,02:00:00,2,...,low,?,quiet,f,closed,variety,U1086,2,0,1
4481,132937,cash,Bar,16:00-23:30;,Sun;,16:00:00,16,0,23:30:00,23,...,low,?,quiet,f,closed,variety,U1003,2,2,1
4482,132937,cash,Bar,16:00-23:30;,Sun;,16:00:00,16,0,23:30:00,23,...,low,?,quiet,f,closed,variety,U1027,1,1,1
4483,132937,cash,Bar,16:00-23:30;,Sun;,16:00:00,16,0,23:30:00,23,...,low,?,quiet,f,closed,variety,U1029,1,1,1


In [104]:
res_rat_filter['Rcuisine'].value_counts()

Series([], Name: count, dtype: int64)

In [34]:
res_rating['Rpayment'].value_counts()

Rpayment
cash                   2898
VISA                   1287
MasterCard-Eurocard    1188
American_Express        588
bank_debit_cards        300
Carte_Blanche            15
Name: count, dtype: int64

In [14]:
rs_st_list = (res_rating['state'].unique())
rs_st_list = rs_st_list.tolist()
rs_st_list.remove('Not Determined')

In [23]:
res_rat_filter  = res_rating[(res_rating['rating'] > 0) & (res_rating['state'] == 'Not Determined')]

In [53]:
ranking['rating'].mean()

1.2521739130434784

In [138]:
ranking = ranking[ranking['state'] == 'San Luis Potosi']
ranking_g= ranking.groupby(['name','price','parking_lot'], as_index=False).aggregate({'rating':'mean','food_rating':'mean','service_rating':'mean'}).sort_values(by=['rating','food_rating','service_rating'], ascending=False).head(10)

max_num = 10 
bars = alt.Chart(ranking_g, title = alt.Title("Top Restaurants based on Rating" )).mark_bar().encode(
    x=alt.X('mean(rating):Q', sort = '-x', title= "Rating"),
    y=alt.Y('name:N',sort=alt.EncodingSortField('rating', op='min', order='descending'), title = "Restaurant Name"),
    color=alt.Color('food_rating',title= "Food Rating"),
    tooltip = ['food_rating','service_rating','price','parking_lot']
    # order = alt.Order(# Sort the segments of the bars by this field
    #   'Avg_Rating',
    #   sort='descending')
) 
text = alt.Chart(ranking_g).mark_text(dx=-15, dy=3, color='white').encode(
    x=alt.X('mean(rating):Q', stack='zero', sort='-x'),
    y=alt.Y('name:N', sort='-x'),
    detail='rating:N',
    text=alt.Text('mean(rating):Q', format='.1f')
)

bars   + text


alt.LayerChart(...)

In [ ]:
# creating Ranking dataframe for rating chart
filt_cols = ['placeID','name' ,'rating','food_rating','service_rating','state','price','parking_lot']
ranking = pd.DataFrame(res_rating,columns=filt_cols)
ranking['Avg_Rating'] = ranking[['rating','food_rating','service_rating']].mean(axis=1)
ranking = ranking[ranking['state'] != 'Not Determined' ]
ranking['parking_lot'] =  ranking['parking_lot'].apply(lambda x: "Not available" if x in 'none' else x)

NameError: name 'sel_st' is not defined

In [210]:

ranking_g= ranking.groupby(['name','price','parking_lot'], as_index=False).aggregate({'rating':'mean','food_rating':'mean','service_rating':'mean'}).sort_values(by=['rating','food_rating','service_rating'], ascending=False)
ranking_c1 = ranking_g[ranking_g['price'] == 'low']
bars = alt.Chart(ranking_c1, title = alt.Title("Top Low priced Restaurants with Parking" )).mark_bar().encode(
    x=alt.X('mean(rating):Q', sort = '-x', title= "Rating"),
    y=alt.Y('name:N',sort=alt.EncodingSortField('rating', op='min', order='descending'), title = "Restaurant Name"),
    color=alt.Color('parking_lot',title= "parking_lot"),

    tooltip = ['food_rating','service_rating','price','parking_lot']
    # order = alt.Order(# Sort the segments of the bars by this field
    #   'Avg_Rating',
    #   sort='descending')
) 
text = alt.Chart(ranking_c1).mark_text(dx=-15, dy=3, color='white').encode(
    x=alt.X('mean(rating):Q', stack='zero', sort='-x'),
    y=alt.Y('name:N', sort='-x'),
    detail='rating:N',
    text=alt.Text('mean(rating):Q', format='.1f')
)
bars + text

KeyError: 'Rpayment'

In [209]:

ranking_g= ranking.groupby(['name','price','parking_lot'], as_index=False).aggregate({'rating':'mean','food_rating':'mean','service_rating':'mean'}).sort_values(by=['rating','food_rating','service_rating'], ascending=False).head(10)

Facet_chart = alt.Chart(ranking_g).mark_bar().encode(
    x=  alt.X('name',title= None),
    y= alt.Y('rating', title="Rating"), color = 'parking_lot'
).properties( title="Restaurants categorized based on their Price", width = 200, height = 200).facet(
        column = alt.Column('price', sort =["low","medium","high"],title= ["Low","Medium","High"], header=alt.Header(title="Restaurants Name", titleOrient= 'bottom'))
).resolve_scale(x='independent')


Facet_chart

alt.FacetChart(...)